# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
// TODO: Replace <FILL IN> with appropriate code

val housing = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("data/housing.csv")

housing = [longitude: double, latitude: double ... 8 more fields]


[longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [2]:
// TODO: Replace <FILL IN> with appropriate code

housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [3]:
// TODO: Replace <FILL IN> with appropriate code

housing.count

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [4]:
// TODO: Replace <FILL IN> with appropriate code

housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [5]:
// TODO: Replace <FILL IN> with appropriate code

housing.filter("population > 10000").count

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [6]:
// TODO: Replace <FILL IN> with appropriate code

housing.describe("housing_median_age", "total_rooms", "median_house_value", "population").show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [7]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age"), min("total_rooms"), avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [8]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").agg(count("ocean_proximity")).show()

+---------------+----------------------+
|ocean_proximity|count(ocean_proximity)|
+---------------+----------------------+
|         ISLAND|                     5|
|     NEAR OCEAN|                  2658|
|       NEAR BAY|                  2290|
|      <1H OCEAN|                  9136|
|         INLAND|                  6551|
+---------------+----------------------+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [9]:
// TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").agg(avg("median_house_value").as("avg_value")).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [10]:
// TODO: Replace <FILL IN> with appropriate code

housing.createOrReplaceTempView("df")
spark.sql("select ocean_proximity, avg(median_house_value) as avg_value from df group by ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [11]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).setOutputCol("features_1")

val housingAttrs = va.transform(housing).select("features_1")

housingAttrs.show(5)

+--------------------+
|          features_1|
+--------------------+
|[41.0,880.0,45260...|
|[21.0,7099.0,3585...|
|[52.0,1467.0,3521...|
|[52.0,1274.0,3413...|
|[52.0,1627.0,3422...|
+--------------------+
only showing top 5 rows



va = vecAssembler_7c41e5ca81a7
housingAttrs = [features_1: vector]


[features_1: vector]

In [12]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features_1").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


coeff = 


1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   

## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [13]:
// TODO: Replace <FILL IN> with appropriate code

val housingCol1 = housing.withColumn("rooms_per_household", expr("total_rooms / households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", expr("total_bedrooms / total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household", expr("population / households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1 = [longitude: double, latitude: double ... 9 more fields]
housingCol2 = [longitude: double, latitude: double ... 10 more fields]
housingExtra = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [14]:
// TODO: Replace <FILL IN> with appropriate code

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")

renamedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [15]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel = label
colCat = ocean_proximity
colNum = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


[longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household]

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [16]:
// TODO: Replace <FILL IN> with appropriate code

renamedHousing.select(renamedHousing.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*).show


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|label|ocean_proximity|rooms_per_household|bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|    0|              0|                  0|              207|                       0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+



As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [17]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(Array("total_bedrooms", "bedrooms_per_room")).setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))                            
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



imputer = imputer_398a9bc20bb3
imputedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [18]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features_")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("features_").setOutputCol("normalized_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|           features_| normalized_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

va = vecAssembler_004741f47c55
featuredHousing = [longitude: double, latitude: double ... 12 more fields]
scaler = stdScal_dd44ccff1b38
scaledHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [19]:
// TODO: Replace <FILL IN> with appropriate code

renamedHousing.agg(countDistinct("ocean_proximity")).show()

+-------------------------------+
|count(DISTINCT ocean_proximity)|
+-------------------------------+
|                              5|
+-------------------------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [20]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol(colCat).setOutputCol("catCol_coded")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|catCol_coded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|         3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082601054|0.15579659106916466|       2.

indexer = strIdx_22530176d375
idxHousing = [longitude: double, latitude: double ... 12 more fields]


[longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [21]:
indexer.fit(renamedHousing).labels

[<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND]

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [22]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("catCol_coded")).setOutputCols(Array("catCol_one_hot_encoded"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------------+----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|catCol_coded|catCol_one_hot_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+------------+----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|         3.0|         (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    113

encoder = oneHotEncoder_5bdd10d12812
ohHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [23]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val pipeline = new Pipeline().setStages(Array(imputer, va, scaler, indexer, encoder))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+------------+----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|           features_| normalized_features|catCol_coded|catCol_one_hot_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+------------+----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.555555555555

pipeline = pipeline_8c83de51e60a
newHousing = [longitude: double, latitude: double ... 15 more fields]


[longitude: double, latitude: double ... 15 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [24]:
// TODO: Replace <FILL IN> with appropriate code

val new_df = newHousing.drop("ocean_proximity")
val va2 = new VectorAssembler().setInputCols(new_df.columns).setOutputCol("features")
val dataset = va2.transform(new_df).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-122.23,37.88,41...|452600.0|
|[-122.22,37.86,21...|358500.0|
|[-122.24,37.85,52...|352100.0|
|[-122.25,37.85,52...|341300.0|
|[-122.25,37.85,52...|342200.0|
+--------------------+--------+
only showing top 5 rows



new_df = [longitude: double, latitude: double ... 14 more fields]
va2 = vecAssembler_cd7bb22af1d1
dataset = [features: vector, label: double]


[features: vector, label: double]

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [25]:
// TODO: Replace <FILL IN> with appropriate code

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet = [features: vector, label: double]
testSet = [features: vector, label: double]


[features: vector, label: double]

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [26]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression()
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}")
println(s"Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-0.4926501197804606,-0.4228217921907426,0.004407897741177998,8.230154852383242E-5,-8.848142825146714E-4,-6.799138293602957E-4,0.0024496255759660548,0.08989014223679068,0.9999930143162868,-0.051287546819942784,-0.7056375984355701,0.012946247970886117,-0.4926501197804606,-0.4228217921907426,0.004407897741177998,8.230154852383242E-5,-8.848142825146714E-4,-6.799138293602957E-4,0.0024496255759660548,0.08989014223679068,-0.051287546819942784,-0.7056375984355701,0.012946247970886117,-0.9870469713818456,-0.903122181691467,0.05547589628884674,0.17955029830187888,-0.371083999993501,-0.7699767096401259,0.9365646526164851,0.17077525850386172,-0.12689425245509314,-0.04071497186473918,0.13446041173032663,159.98488460441948,31.071532196386265,-127.17778875664808,-289.4725719624741,-451.7093189791858]
Intercept: -161.7078007038805
RMSE: 9.583462178442572


lr = linReg_d6e5a7562229
lrModel = linReg_d6e5a7562229
trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@43b48ecc


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@43b48ecc

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [27]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|103598.59298259343|103600.0|[-124.3,41.84,17....|
| 50800.46914093848| 50800.0|[-124.23,40.81,52...|
| 69000.59695064677| 69000.0|[-124.19,40.77,30...|
| 70200.33255731777| 70200.0|[-124.18,40.78,37...|
| 70500.12698294852| 70500.0|[-124.18,40.79,39...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 9.534925148650462


predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_7656beb27367
rmse = 9.534925148650462


9.534925148650462

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [28]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setLabelCol("label")
    .setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+-----------------+--------+--------------------+
|       prediction|   label|            features|
+-----------------+--------+--------------------+
|96128.65384615384|103600.0|[-124.3,41.84,17....|
|51195.95576923077| 50800.0|[-124.23,40.81,52...|
|69676.24633431085| 69000.0|[-124.19,40.77,30...|
|69676.24633431085| 70200.0|[-124.18,40.78,37...|
|69676.24633431085| 70500.0|[-124.18,40.79,39...|
+-----------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 6557.264526161531


dt = dtr_1787176febcd
dtModel = DecisionTreeRegressionModel (uid=dtr_1787176febcd) of depth 5 with 61 nodes
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_6713d475d8b8
rmse = 6557.264526161531


6557.264526161531

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [29]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setLabelCol("label")
    .setFeaturesCol("features")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 148098.7051516168|103600.0|[-124.3,41.84,17....|
|158167.30447021744| 50800.0|[-124.23,40.81,52...|
|138721.10524924428| 69000.0|[-124.19,40.77,30...|
|125228.08416821228| 70200.0|[-124.18,40.78,37...|
|120618.81451048236| 70500.0|[-124.18,40.79,39...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 24709.631173970232


rf = rfr_e240f163d144
rfModel = RandomForestRegressionModel (uid=rfr_e240f163d144) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_9eaa37b8ab07
rmse = 24709.631173970232


24709.631173970232

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [30]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setLabelCol("label")
    .setFeaturesCol("features")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 99551.10805526827|103600.0|[-124.3,41.84,17....|
| 50834.03633630346| 50800.0|[-124.23,40.81,52...|
|65368.395888418985| 69000.0|[-124.19,40.77,30...|
| 65325.49860180558| 70200.0|[-124.18,40.78,37...|
|  75331.9154788102| 70500.0|[-124.18,40.79,39...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 5876.655157695818


gb = gbtr_2ebeae2e0758
gbModel = GBTRegressionModel (uid=gbtr_2ebeae2e0758) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_34888b670e2a
rmse = 5876.655157695818


5876.655157695818

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [56]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
                .addGrid(rf.numTrees, Array(1, 5, 10))
                .addGrid(rf.maxDepth, Array(5, 10, 15))
                .build() 

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val cv = new CrossValidator()
        .setEvaluator(evaluator)
        .setEstimator(rf)
        .setEstimatorParamMaps(paramGrid)
        .setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predparamGridictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
// val va2 = new VectorAssembler().setInputCols(new_df.columns).setOutputCol("features")
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:998)
  at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1304)
  at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:202)
  at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:218)
  at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:153)
  at org.apache.spark.util.ThreadUtils$.awaitReady(ThreadUtils.scala:222)
  at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:633)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:2034)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:2055)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:2074)
  at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:363)
  at org.apache.spark.sql.exec

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [57]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "Input column 1")
    final val inputCol2 = new Param[String](this, "inputCol2", "Input column 2")
    final val outputCol = new Param[String](this, "outputCol", "Output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    // Check that the input types are strings
    for (colName <- Array(inputCol1, inputCol2)) {
        val idx = schema.fieldIndex($(colName))
        val field = schema.fields(idx)
        if (field.dataType != DoubleType) {
          throw new Exception(s"Input type ${field.dataType} did not match input type DoubleType")
        }
    }
     // Add the return field
    schema.add(StructField($(outputCol), DoubleType, false)) 
  }
}


defined trait MyParams


lastException: Throwable = null


Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [58]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.sql.functions.{col, udf}

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap): MyTransformer = {
        defaultCopy(extra)
    }

    override def transformSchema(schema: StructType) : StructType = {
    // Check that the input types are strings
    for (colName <- Array(inputCol1, inputCol2)) {
        val idx = schema.fieldIndex($(colName))
        val field = schema.fields(idx)
        if (field.dataType != DoubleType) {
          throw new Exception(s"Input type ${field.dataType} did not match input type DoubleType")
        }
    }
     // Add the return field
    schema.add(StructField($(outputCol), DoubleType, false)) 
  }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        val divide = udf {(x1: Double, x2: Double) => x1/x2 }
        dataset.select(col("*"), divide(dataset.col($(inputCol1)), dataset.col($(inputCol2))).as($(outputCol)))
  }
}

defined class MyTransformer


Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [59]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

Name: org.apache.spark.SparkException
Message: Job 1020 cancelled as part of cancellation of all jobs
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1602)
  at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1542)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply$mcVI$sp(DAGScheduler.scala:735)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:735)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:735)
  at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
  at org.apache.spark.scheduler.DAGScheduler.doCancelAllJobs(DAGScheduler.scala:735)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1795)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1772

---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [60]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

defined class Instance
defined object Helper


lastException: Throwable = null


In [61]:
class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

Name: Unknown Error
Message: <console>:18: error: not found: type RDD
         def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
                                      ^
<console>:19: error: not found: type RegressionMetrics
           val regressionMetrics = new RegressionMetrics(labelsAndPreds)
                                       ^
<console>:23: error: type Vector takes type parameters
         def gradientSummand(weights: Vector, lp: Instance): Vector = {
                                                             ^
<console>:23: error: type Vector takes type parameters
         def gradientSummand(weights: Vector, lp: Instance): Vector = {
                                      ^
<console>:26: error: not found: value Vectors
           return Vectors.dense(seq.toArray)
                  ^
<console>:29: error: type Vector takes type parameters
         def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
                    

In [62]:
abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

Name: Unknown Error
Message: <console>:16: error: not found: type PredictionModel
         extends PredictionModel[FeaturesType, Model] {
                 ^
<console>:20: error: type Vector takes type parameters
           extends MyLinearModel[Vector, MyLinearModelImpl] {
                                 ^
<console>:19: error: type Vector takes type parameters
       class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
                                                                      ^
<console>:22: error: not found: type ParamMap
         override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)
                                  ^
<console>:22: error: not found: value defaultCopy
         override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)
                                                                 ^
<console>:24: error: type Vector takes type parameters
         def predict(features:

In [63]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

Name: Unknown Error
Message: <console>:55: error: not found: type MyLinearModel
           Model <: MyLinearModel[FeaturesType, Model]]
                    ^
<console>:60: error: not found: type MyLinearModelImpl
           extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
                                                                      ^
<console>:65: error: not found: type MyLinearModelImpl
         def train(dataset: Dataset[_]): MyLinearModelImpl = {
                                         ^
<console>:76: error: not found: type LR
           val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)
                                              ^
<console>:78: error: not found: type MyLinearModelImpl
           new MyLinearModelImpl(uid, weights, trainingError)
               ^

StackTrace: 

In [64]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Name: Unknown Error
Message: <console>:178: error: not found: type MyLinearRegressionImpl
       val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
                    ^

StackTrace: 

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

In [155]:
// Read the data
val ccdata = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("data/ccdefault.csv")

ccdata = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

In [156]:
// Print the schema
ccdata.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

In [157]:
ccdata.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+------------------+--------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+----------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+-------------------+
|summary|               ID|         LIMIT_BAL|               SEX|         EDUCATION|          MARRIAGE|              AGE|             PAY_0|               PAY_2|             PAY_3|               PAY_4|             PAY_5|            PAY_6|        BILL_AMT1|        BILL_AMT2|        BILL_AMT3|         BILL_AMT4|        BILL_AMT5|       BILL_AMT6|         PAY_AMT1|          PAY_AMT2|         PAY_AMT3|          PAY_AMT4|          PAY_AMT5|         PAY_AMT6|            DEFAULT|
+-------+-----------------+-----------------

In [158]:
ccdata.count

30000

In [159]:
// Look at the data
ccdata.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    

In [160]:
// Remove IDs 
val cleanCCdata = ccdata.drop("ID")

cleanCCdata = [LIMIT_BAL: int, SEX: int ... 22 more fields]


[LIMIT_BAL: int, SEX: int ... 22 more fields]

In [161]:
// Divide into numerical and categorical data

// label columns
val colLabel = "DEFAULT"

// categorical columns
val colCat = Array("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6")

// numerical columns
val colNum = cleanCCdata.columns.filter(_ != colLabel).filter(!colCat.contains(_))

colLabel = DEFAULT
colCat = Array(SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)
colNum = Array(LIMIT_BAL, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)


[LIMIT_BAL, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6]

In [162]:
// Clean data? // check for null or nan etc....?
cleanCCdata.select(cleanCCdata.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*).show


+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|        0|  0|        0|       0|  0|    0|    0|    0|    0|    0|    0|        0|        0|        0|        0|        0|        0|       0|       0|       0|       0|       0|       0|      0|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+



In [163]:
// Check distributions of some numerical features
cleanCCdata.describe("LIMIT_BAL", "AGE", "BILL_AMT1", "PAY_AMT1").show()



+-------+------------------+-----------------+-----------------+-----------------+
|summary|         LIMIT_BAL|              AGE|        BILL_AMT1|         PAY_AMT1|
+-------+------------------+-----------------+-----------------+-----------------+
|  count|             30000|            30000|            30000|            30000|
|   mean|167484.32266666667|          35.4855|       51223.3309|        5663.5805|
| stddev|129747.66156720246|9.217904068090155|73635.86057552966|16563.28035402577|
|    min|             10000|               21|          -165580|                0|
|    max|           1000000|               79|           964511|           873552|
+-------+------------------+-----------------+-----------------+-----------------+



In [164]:
cleanCCdata.describe("SEX", "EDUCATION", "MARRIAGE", "PAY_0").show()


+-------+------------------+------------------+------------------+------------------+
|summary|               SEX|         EDUCATION|          MARRIAGE|             PAY_0|
+-------+------------------+------------------+------------------+------------------+
|  count|             30000|             30000|             30000|             30000|
|   mean|1.6037333333333332|1.8531333333333333|1.5518666666666667|           -0.0167|
| stddev|0.4891291960902602|0.7903486597207269|0.5219696006132467|1.1238015279973335|
|    min|                 1|                 0|                 0|                -2|
|    max|                 2|                 6|                 3|                 8|
+-------+------------------+------------------+------------------+------------------+



In [165]:
// Scale numerical variables

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features_")
val featuredCcdata = va.transform(cleanCCdata)

val scaler = new StandardScaler().setInputCol("features_").setOutputCol("normalized_features")
val scaledCcdata = scaler.fit(featuredCcdata).transform(featuredCcdata)

scaledCcdata.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+--------------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|           features_| normalized_features|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+--------------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|(14,[0,1,2,3,4,9]...|(14,[0,1,2,3,4,9]...|
|   120000|  2|        2|       2| 26|   -1|

va = vecAssembler_2f08a9877cf1
featuredCcdata = [LIMIT_BAL: int, SEX: int ... 23 more fields]
scaler = stdScal_8a69f8907b2b
scaledCcdata = [LIMIT_BAL: int, SEX: int ... 24 more fields]


[LIMIT_BAL: int, SEX: int ... 24 more fields]

In [166]:
// Create dummy variables

import org.apache.spark.ml.feature.OneHotEncoderEstimator

val dummy_cols = Array("SEX_D", "EDUCATION_D", "MARRIAGE_D")
val encoder = new OneHotEncoderEstimator().setInputCols(Array("SEX", "EDUCATION", "MARRIAGE")).setOutputCols(dummy_cols)
val data = encoder.fit(scaledCcdata).transform(scaledCcdata)

data.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+--------------------+-------------+-------------+-------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|           features_| normalized_features|        SEX_D|  EDUCATION_D|   MARRIAGE_D|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+--------------------+-------------+-------------+-------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|     

dummy_cols = Array(SEX_D, EDUCATION_D, MARRIAGE_D)
encoder = oneHotEncoder_cd3d0717d3aa
data = [LIMIT_BAL: int, SEX: int ... 27 more fields]


[LIMIT_BAL: int, SEX: int ... 27 more fields]

In [167]:
//Creating training and test set

val va2 = new VectorAssembler().setInputCols(Array("features_", "normalized_features", "SEX_D", "EDUCATION_D", "MARRIAGE_D", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6")).setOutputCol("features")
val dataset_intermediate = va2.transform(data).select("features", "DEFAULT")
val dataset = dataset_intermediate.withColumnRenamed("DEFAULT", "label")

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    1|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
|(45,[0,1,2,3,4,5,...|    0|
+--------------------+-----+
only showing top 10 rows



va2 = vecAssembler_325c82717b20
dataset_intermediate = [features: vector, DEFAULT: int]
dataset = [features: vector, label: int]
trainSet = [features: vector, label: int]
testSet = [features: vector, label: int]


[features: vector, label: int]

In [1]:
// Logistic Regression: Build, train and predict. 
import org.apache.spark.ml.classification.{BinaryLogisticRegressionSummary, LogisticRegression}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.regression.LabeledPoint

// train the model
val lr = new LogisticRegression().setFamily("multinomial")
val lrModel = lr.fit(trainSet)


val evaluator = new BinaryClassificationEvaluator()
val predictions = lrModel.transform(testSet)

val accuracy = evaluator.evaluate(predictions)

val rfAuc = evaluator.evaluate(predictions)
println(s"Area Under ROC (AUC) on test data = $rfAuc")

Area Under ROC (AUC) on test data = 0.7211899016395156


lr = logreg_f524eeb04fa6
lrModel = logreg_f524eeb04fa6
evaluator = binEval_7a9b9a4e0ce9
predictions = [features: vector, label: int ... 3 more fields]
accuracy = 0.7211899016395156
rfAuc = 0.7211899016395156


0.7211899016395156

In [2]:
// Logistic Regression Cross Validation
val paramGrid = new ParamGridBuilder()
                .addGrid(lr.regParam, Array(0.1, 0.3, 0.5))
                .build() 


val evaluator = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction")
val cv = new CrossValidator()
        .setEvaluator(evaluator)
        .setEstimator(lr)
        .setEstimatorParamMaps(paramGrid)
        .setNumFolds(3)

val cvModel = cv.fit(trainSet)
val predictions = cvModel.transform(testSet)

predictions.select("prediction", "label", "features").show(5)

val lrCvAuc = evaluator.evaluate(predictions)
println(s"Area Under ROC (AUC) cross validated on test data = $rfAuc")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) cross validated on test data = 0.7211899016395156


paramGrid = 
evaluator = mcEval_ac9fe3516dd6
cv = cv_40f7c9b8d4f5
cvModel = cv_40f7c9b8d4f5
predictions = [features: vector, label: int ... 3 more fields]
lrCvAuc = 0.7480134670765274


Array({
	logreg_52f1242c72e5-regParam: 0.1
}, {
	logreg_52f1242c72e5-regParam: 0.3
}, {
	logreg_52f1242c72e5-regParam: 0.5
})


0.7480134670765274

In [3]:
// Decision Tree: Build, train and predict. 

import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val dt = new DecisionTreeClassifier().setLabelCol("label")
    .setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val m_evaluator = new MulticlassClassificationEvaluator().setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val b_evaluator = new BinaryClassificationEvaluator()

val accuracy = m_evaluator.evaluate(predictions)

val rfAuc = b_evaluator.evaluate(predictions)
println(s"Area Under ROC (AUC) on test data = $rfAuc")
println(s"Test Error = ${1 - accuracy}")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.3358848018723757
Test Error = 0.17887134699361773


dt = dtc_ebe54942a50d
dtModel = DecisionTreeClassificationModel (uid=dtc_ebe54942a50d) of depth 5 with 63 nodes
predictions = [features: vector, label: int ... 3 more fields]
m_evaluator = mcEval_5ce1424def89
b_evaluator = binEval_7abfe81aad0b
accuracy = 0.8211286530063823
rfAuc = 0.3358848018723757


0.3358848018723757

In [6]:
// Decision Tree Cross Validation
val paramGrid = new ParamGridBuilder()
                .addGrid(dt.maxDepth, Array(5, 10, 15))                
                .build() 


val evaluator = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction")
val cv = new CrossValidator()
        .setEvaluator(evaluator)
        .setEstimator(dt)
        .setEstimatorParamMaps(paramGrid)
        .setNumFolds(3)

val cvModel = cv.fit(trainSet)
val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val dtCvAuc = evaluator.evaluate(predictions)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows



paramGrid = 
evaluator = mcEval_5ab8f370dea1
cv = cv_ea3b46a92f76
cvModel = cv_ea3b46a92f76
predictions = [features: vector, label: int ... 3 more fields]
dtCvAuc = 0.7954467448560079


Array({
	dtc_ebe54942a50d-maxDepth: 5
}, {
	dtc_ebe54942a50d-maxDepth: 10
}, {
	dtc_ebe54942a50d-maxDepth: 15
})


0.7954467448560079

In [5]:
// Random Forest: Build, train and predict. 

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val rf = new RandomForestClassifier().setLabelCol("label")
    .setFeaturesCol("features")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val m_evaluator = new MulticlassClassificationEvaluator().setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val b_evaluator = new BinaryClassificationEvaluator()

val accuracy = m_evaluator.evaluate(predictions)

val rfAuc = b_evaluator.evaluate(predictions)
println(s"Area Under ROC (AUC) on test data = $rfAuc")
println(s"Test Error = ${1 - accuracy}")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.7798707887726134
Test Error = 0.1822304333221364


rf = rfc_979f3bf7fbac
rfModel = RandomForestClassificationModel (uid=rfc_979f3bf7fbac) with 20 trees
predictions = [features: vector, label: int ... 3 more fields]
m_evaluator = mcEval_7f8a2e312e2b
b_evaluator = binEval_a7646a5f25e6
accuracy = 0.8177695666778636
rfAuc = 0.7798707887726134


0.7798707887726134

In [7]:
val paramGrid = new ParamGridBuilder()
                .addGrid(rf.numTrees, Array(1, 5, 10))
                .addGrid(rf.maxDepth, Array(5, 10, 15))
                .build() 

val evaluator = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction")
val cv = new CrossValidator()
        .setEvaluator(evaluator)
        .setEstimator(rf)
        .setEstimatorParamMaps(paramGrid)
        .setNumFolds(3)

val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rfCvAuc = evaluator.evaluate(predictions)


+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
|       0.0|    0|(45,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows



paramGrid = 
evaluator = mcEval_39d6f5b818b2


Array({
	rfc_979f3bf7fbac-maxDepth: 5,
	rfc_979f3bf7fbac-numTrees: 1
}, {
	rfc_979f3bf7fbac-maxDepth: 5,
	rfc_979f3bf7fbac-numTrees: 5
}, {
	rfc_979f3bf7fbac-maxDepth: 5,
	rfc_979f3bf7fbac-numTrees: 10
}, {
	rfc_979f3bf7fbac-maxDepth: 10,
	rfc_979f3bf7fbac-numTrees: 1
}, {
	rfc_979f3bf7fbac-maxDepth: 10,
	rfc_979f3bf7fbac-numTrees: 5
}, {
	rfc_979f3bf7fbac-maxDepth: 10,
	rfc_979f3bf7fbac-numTrees: 10
}, {
	rfc_979f3bf7fbac-maxDepth: 15,
	rfc_979f3bf7fbac-numTrees: 1
}, {
	rfc_979f3bf7fbac-maxDepth: 15,
	rfc_979f3bf7fbac-numTrees: 5
}, {
	rfc_979f3bf7fbac-maxDepth: 15,
	rfc_979f3bf7fbac-numTrees: 10
})
cv: org.apache.spark.ml.tuning.Cros...


mcEval_39d6f5b818b2

In [8]:
// Comparison
println(s"Logistic Regression: Accuracy on test data = $lrCvAuc")

println(s"Decision Tree: Accuracy on test data = $dtCvAuc")

println(s"Random Forest: Accuracy on test data = $rfCvAuc")


Logistic Regression: Accuracy on test data = 0.7480134670765274
Decision Tree: Accuracy on test data = 0.7954467448560079
Random Forest: Accuracy on test data = 0.8008211152919924


## Choosing model 

Given these results we would chose model ...